## 1. Importar Librerías

In [13]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score,
    precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc
)
from sklearn.preprocessing import label_binarize

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, Model
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.utils import to_categorical

import warnings
warnings.filterwarnings('ignore')

# Configuración de reproducibilidad
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU disponible: {tf.config.list_physical_devices('GPU')}")

TensorFlow version: 2.20.0
GPU disponible: []


## 2. Cargar y Preparar Datos Tabulares

In [14]:
# Cargar datos tabulares
df = pd.read_csv('./data/animal_disease_prediction_cleaned.csv')

print(f"Forma del dataset: {df.shape}")
print(f"\nClases de enfermedades: {df['Disease_Prediction'].nunique()}")
print(f"\nDistribución de clases:")
print(df['Disease_Prediction'].value_counts())

Forma del dataset: (44100, 22)

Clases de enfermedades: 31

Distribución de clases:
Disease_Prediction
Parvovirus               7500
Flu                      4200
Distemper                3600
Cough                    3300
Gastroenteritis          2700
Respiratory Infection    2700
Leukemia                 2400
Peritonitis              2400
Leptospirosis            2100
Herpes                   2100
Pancreatitis             1200
Hepatitis                1200
Panleukopenia             900
Asthma                    900
Intestinal Parasites      600
Lyme Disease              600
Fungal Infection          600
Respiratory Disease       600
Heartworm                 600
Chlamydia                 600
Arthritis                 300
Ringworm                  300
Tick-Borne Disease        300
Kidney Disease            300
Allergic Rhinitis         300
IBD                       300
Bronchitis                300
Conjunctivitis            300
Hyperthyroidism           300
Coronavirus               3

In [15]:
# Mapear enfermedades a las categorías de imágenes disponibles
disease_to_image_class = {
    'Parvovirus': 'Parvovirus in Dog',
    'Respiratory Infection': 'Eye Infection in Cat',
    'Gastroenteritis': 'Parvovirus in Dog',
    'Fungal Infection': 'Fungal Infection in Cat',
    'Lyme Disease': 'Tick Infestation in Dog',
    'Intestinal Parasites': 'Worm Infection in Cat',
    'Distemper': 'Distemper in Dog',
    'Cough': 'Kennel Cough in Dog',
    'Ringworm': 'Ringworm in Cat',
    'Tick-Borne Disease': 'Tick Infestation in Dog',
    'Herpes': 'Eye Infection in Cat',
    'Leukemia': 'Feline Leukemia',
    'Heartworm': 'Kennel Cough in Dog',
    'Peritonitis': 'Feline Panleukopenia',
    'Conjunctivitis': 'Eye Infection in Cat',
    'Bronchitis': 'Kennel Cough in Dog',
    'Flu': 'Eye Infection in Cat',
    'Pancreatitis': 'Worm Infection in Cat',
    'IBD': 'Worm Infection in Cat',
    'Allergic Rhinitis': 'Skin Allergy in Dog',
    'Kidney Disease': 'Urinary Tract Infection in Cat',
    'Arthritis': 'Hot Spots in Dog',
    'Chlamydia': 'Eye Infection in Cat',
    'Coronavirus': 'Feline Panleukopenia',
    'Asthma': 'Eye Infection in Cat',
    'Hyperthyroidism': 'Feline Leukemia',
    'Hepatitis': 'Parvovirus in Dog',
    'Leptospirosis': 'Parvovirus in Dog',
    'FIV': 'Feline Leukemia'
}

df['Image_Class'] = df['Disease_Prediction'].map(disease_to_image_class)

# Filtrar solo las enfermedades que tienen mapeo a imágenes
df = df[df['Image_Class'].notna()].reset_index(drop=True)

print(f"Datos después del filtrado: {df.shape}")
print(f"\nClases únicas de imágenes: {df['Image_Class'].nunique()}")

Datos después del filtrado: (42600, 23)

Clases únicas de imágenes: 13


In [16]:
# Preparar características tabulares
# Codificar variables categóricas
label_encoders = {}
categorical_cols = ['Animal_Type', 'Breed', 'Gender', 'Symptom_1', 'Symptom_2', 'Symptom_3', 'Symptom_4']

for col in categorical_cols:
    le = LabelEncoder()
    df[col + '_encoded'] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le

# Seleccionar características para el modelo
feature_cols = [
    'Animal_Type_encoded', 'Breed_encoded', 'Age', 'Gender_encoded', 'Weight',
    'Symptom_1_encoded', 'Symptom_2_encoded', 'Symptom_3_encoded', 'Symptom_4_encoded',
    'Appetite_Loss', 'Vomiting', 'Diarrhea', 'Coughing', 'Labored_Breathing',
    'Lameness', 'Skin_Lesions', 'Nasal_Discharge', 'Eye_Discharge',
    'Body_Temperature', 'Heart_Rate', 'Duration_days'
]

X_tabular = df[feature_cols].values

# Codificar etiquetas de enfermedades
disease_encoder = LabelEncoder()
y_labels = disease_encoder.fit_transform(df['Disease_Prediction'])
num_classes = len(disease_encoder.classes_)

print(f"\nNúmero de características tabulares: {X_tabular.shape[1]}")
print(f"Número de clases: {num_classes}")
print(f"Clases: {disease_encoder.classes_}")


Número de características tabulares: 21
Número de clases: 29
Clases: ['Allergic Rhinitis' 'Arthritis' 'Asthma' 'Bronchitis' 'Chlamydia'
 'Conjunctivitis' 'Coronavirus' 'Cough' 'Distemper' 'FIV' 'Flu'
 'Fungal Infection' 'Gastroenteritis' 'Heartworm' 'Hepatitis' 'Herpes'
 'Hyperthyroidism' 'IBD' 'Intestinal Parasites' 'Kidney Disease'
 'Leptospirosis' 'Leukemia' 'Lyme Disease' 'Pancreatitis' 'Parvovirus'
 'Peritonitis' 'Respiratory Infection' 'Ringworm' 'Tick-Borne Disease']


## 3. Preparar Datos de Imágenes

In [17]:
# Configuración de imágenes
IMG_SIZE = (224, 224)
images_base_path = './data/images'

# En lugar de cargar todas las imágenes en memoria, 
# guardaremos solo las rutas y las cargaremos bajo demanda

print("Preparando referencias a imágenes...")
image_paths = []
valid_indices = []

for idx, image_class in enumerate(df['Image_Class']):
    class_path = os.path.join(images_base_path, image_class)
    
    if not os.path.exists(class_path):
        continue
    
    images_files = [f for f in os.listdir(class_path) 
                   if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    if not images_files:
        continue
    
    # Seleccionar imagen aleatoria y guardar su ruta
    img_file = np.random.choice(images_files)
    img_path = os.path.join(class_path, img_file)
    image_paths.append(img_path)
    valid_indices.append(idx)
    
    if (idx + 1) % 1000 == 0:
        print(f"Procesadas {idx + 1}/{len(df)} muestras")

# Filtrar datos tabulares y etiquetas
X_tabular_filtered = X_tabular[valid_indices]
y_labels_filtered = y_labels[valid_indices]

print(f"\nTotal de muestras válidas: {len(image_paths)}")
print(f"Forma de datos tabulares: {X_tabular_filtered.shape}")
print(f"Forma de etiquetas: {y_labels_filtered.shape}")

Preparando referencias a imágenes...
Procesadas 1000/42600 muestras
Procesadas 1000/42600 muestras
Procesadas 2000/42600 muestras
Procesadas 2000/42600 muestras
Procesadas 3000/42600 muestras
Procesadas 3000/42600 muestras
Procesadas 4000/42600 muestras
Procesadas 4000/42600 muestras
Procesadas 5000/42600 muestras
Procesadas 5000/42600 muestras
Procesadas 6000/42600 muestras
Procesadas 6000/42600 muestras
Procesadas 7000/42600 muestras
Procesadas 7000/42600 muestras
Procesadas 8000/42600 muestras
Procesadas 8000/42600 muestras
Procesadas 9000/42600 muestras
Procesadas 9000/42600 muestras
Procesadas 10000/42600 muestras
Procesadas 10000/42600 muestras
Procesadas 11000/42600 muestras
Procesadas 11000/42600 muestras
Procesadas 12000/42600 muestras
Procesadas 12000/42600 muestras
Procesadas 13000/42600 muestras
Procesadas 13000/42600 muestras
Procesadas 14000/42600 muestras
Procesadas 14000/42600 muestras
Procesadas 15000/42600 muestras
Procesadas 15000/42600 muestras
Procesadas 16000/4260

## 4. División de Datos y Normalización

In [18]:
# Dividir datos en entrenamiento, validación y prueba
# Primero separar test set (15%)
img_paths_temp, img_paths_test, X_tab_temp, X_tab_test, y_temp, y_test = train_test_split(
    image_paths, X_tabular_filtered, y_labels_filtered, test_size=0.15, random_state=42, stratify=y_labels_filtered
)

# Luego dividir entrenamiento y validación (70% train, 15% val)
img_paths_train, img_paths_val, X_tab_train, X_tab_val, y_train, y_val = train_test_split(
    img_paths_temp, X_tab_temp, y_temp, test_size=0.176, random_state=42, stratify=y_temp
)

print("División de datos:")
print(f"Entrenamiento: {len(img_paths_train)} muestras ({len(img_paths_train)/len(image_paths)*100:.1f}%)")
print(f"Validación: {len(img_paths_val)} muestras ({len(img_paths_val)/len(image_paths)*100:.1f}%)")
print(f"Prueba: {len(img_paths_test)} muestras ({len(img_paths_test)/len(image_paths)*100:.1f}%)")

# Normalizar datos tabulares
scaler = StandardScaler()
X_tab_train = scaler.fit_transform(X_tab_train)
X_tab_val = scaler.transform(X_tab_val)
X_tab_test = scaler.transform(X_tab_test)

# Convertir etiquetas a categorical
y_train_cat = to_categorical(y_train, num_classes)
y_val_cat = to_categorical(y_val, num_classes)
y_test_cat = to_categorical(y_test, num_classes)

print(f"\nForma de y_train_cat: {y_train_cat.shape}")

División de datos:
Entrenamiento: 29837 muestras (70.0%)
Validación: 6373 muestras (15.0%)
Prueba: 6390 muestras (15.0%)

Forma de y_train_cat: (29837, 29)


In [19]:
# Crear generador de datos personalizado para modelo multimodal
class MultimodalDataGenerator(keras.utils.Sequence):
    """Generador que carga imágenes bajo demanda y combina con datos tabulares"""
    
    def __init__(self, image_paths, tabular_data, labels, batch_size=32, 
                 img_size=(224, 224), shuffle=True, augment=False):
        self.image_paths = image_paths
        self.tabular_data = tabular_data
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        self.augment = augment
        self.indexes = np.arange(len(self.image_paths))
        
        # Data augmentation (solo para entrenamiento)
        if self.augment:
            self.datagen = ImageDataGenerator(
                rotation_range=15,
                width_shift_range=0.1,
                height_shift_range=0.1,
                zoom_range=0.1,
                horizontal_flip=True,
                fill_mode='nearest'
            )
        
        self.on_epoch_end()
    
    def __len__(self):
        """Número de batches por época"""
        return int(np.ceil(len(self.image_paths) / self.batch_size))
    
    def __getitem__(self, index):
        """Genera un batch de datos"""
        # Obtener índices del batch
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        
        # Cargar y preparar datos del batch
        batch_images = np.zeros((len(indexes), *self.img_size, 3), dtype=np.float32)
        batch_tabular = self.tabular_data[indexes].astype(np.float32)
        batch_labels = self.labels[indexes].astype(np.float32)
        
        # Cargar imágenes
        for i, idx in enumerate(indexes):
            try:
                img = load_img(self.image_paths[idx], target_size=self.img_size)
                img_array = img_to_array(img) / 255.0
                
                # Aplicar augmentation si está habilitado
                if self.augment:
                    img_array = self.datagen.random_transform(img_array)
                
                batch_images[i] = img_array
            except Exception as e:
                print(f"Error cargando imagen {self.image_paths[idx]}: {e}")
                # Usar imagen en blanco si falla la carga
                batch_images[i] = np.zeros((*self.img_size, 3))
        
        # Retornar como diccionario para compatibilidad con Keras 3.x
        return {'image_input': batch_images, 'tabular_input': batch_tabular}, batch_labels
    
    def on_epoch_end(self):
        """Mezclar índices al final de cada época"""
        if self.shuffle:
            np.random.shuffle(self.indexes)

# Crear generadores
BATCH_SIZE = 32

print("Creando generadores de datos...")
train_generator = MultimodalDataGenerator(
    img_paths_train, X_tab_train, y_train_cat,
    batch_size=BATCH_SIZE, shuffle=True, augment=True
)

val_generator = MultimodalDataGenerator(
    img_paths_val, X_tab_val, y_val_cat,
    batch_size=BATCH_SIZE, shuffle=False, augment=False
)

test_generator = MultimodalDataGenerator(
    img_paths_test, X_tab_test, y_test_cat,
    batch_size=BATCH_SIZE, shuffle=False, augment=False
)

print(f"\n✓ Generadores creados exitosamente:")
print(f"  - Train: {len(train_generator)} batches")
print(f"  - Val: {len(val_generator)} batches")
print(f"  - Test: {len(test_generator)} batches")

Creando generadores de datos...

✓ Generadores creados exitosamente:
  - Train: 933 batches
  - Val: 200 batches
  - Test: 200 batches


## 5. Construcción del Modelo Multimodal

In [20]:
def create_multimodal_model(img_shape, tabular_shape, num_classes):
    """
    Crea un modelo multimodal que combina:
    - Branch de CNN para imágenes (EfficientNetB3)
    - Branch de red densa para datos tabulares
    - Fusión por concatenación
    - Clasificador final
    """
    
    # ========== BRANCH DE IMÁGENES ==========
    # Crear input de imagen
    input_img = layers.Input(shape=img_shape, name='image_input')
    
    # SOLUCIÓN: Usar EfficientNetV2 o ResNet como alternativa
    # Si EfficientNetB3 falla, usamos una arquitectura compatible
    try:
        from tensorflow.keras.applications import EfficientNetV2B3
        print("Intentando cargar EfficientNetV2B3...")
        base_cnn_model = EfficientNetV2B3(
            include_top=False,
            weights='imagenet',
            input_shape=img_shape,
            pooling='avg'
        )
        print("✓ EfficientNetV2B3 con pesos de ImageNet cargado exitosamente")
    except Exception as e1:
        print(f"⚠ EfficientNetV2 no disponible: {e1}")
        try:
            from tensorflow.keras.applications import ResNet50
            print("Intentando cargar ResNet50 como alternativa...")
            base_cnn_model = ResNet50(
                include_top=False,
                weights='imagenet',
                input_shape=img_shape,
                pooling='avg'
            )
            print("✓ ResNet50 con pesos de ImageNet cargado exitosamente")
        except Exception as e2:
            print(f"⚠ ResNet50 también falló: {e2}")
            print("Usando EfficientNetB3 sin pesos pre-entrenados...")
            base_cnn_model = EfficientNetB3(
                include_top=False,
                weights=None,
                input_shape=img_shape,
                pooling='avg'
            )
    
    # Aplicar el CNN a la entrada
    x_img = base_cnn_model(input_img)
    
    # Congelar capas iniciales (fine-tuning)
    trainable_layers = min(50, len(base_cnn_model.layers))
    for layer in base_cnn_model.layers[:-trainable_layers]:
        layer.trainable = False
    
    print(f"Capas entrenables en CNN: {sum([1 for layer in base_cnn_model.layers if layer.trainable])}/{len(base_cnn_model.layers)}")
    
    # Capas adicionales para imagen
    x_img = layers.Dropout(0.3, name='img_dropout_1')(x_img)
    x_img = layers.Dense(512, activation='relu', name='img_dense_1')(x_img)
    x_img = layers.BatchNormalization(name='img_bn_1')(x_img)
    x_img = layers.Dropout(0.3, name='img_dropout_2')(x_img)
    img_embedding = layers.Dense(256, activation='relu', name='img_embedding')(x_img)
    
    # ========== BRANCH TABULAR ==========
    input_tab = layers.Input(shape=(tabular_shape,), name='tabular_input')
    
    x_tab = layers.Dense(128, activation='relu', name='tab_dense_1')(input_tab)
    x_tab = layers.BatchNormalization(name='tab_bn_1')(x_tab)
    x_tab = layers.Dropout(0.4, name='tab_dropout_1')(x_tab)
    x_tab = layers.Dense(64, activation='relu', name='tab_dense_2')(x_tab)
    x_tab = layers.BatchNormalization(name='tab_bn_2')(x_tab)
    x_tab = layers.Dropout(0.4, name='tab_dropout_2')(x_tab)
    tab_embedding = layers.Dense(128, activation='relu', name='tab_embedding')(x_tab)
    
    # ========== FUSIÓN MULTIMODAL ==========
    # Concatenación de embeddings
    fused = layers.Concatenate(name='fusion')([img_embedding, tab_embedding])
    
    # ========== META-MODELO (Clasificador Final) ==========
    x = layers.Dense(256, activation='relu', name='fusion_dense_1')(fused)
    x = layers.BatchNormalization(name='fusion_bn_1')(x)
    x = layers.Dropout(0.4, name='fusion_dropout_1')(x)
    x = layers.Dense(128, activation='relu', name='fusion_dense_2')(x)
    x = layers.BatchNormalization(name='fusion_bn_2')(x)
    x = layers.Dropout(0.3, name='fusion_dropout_2')(x)
    
    # Capa de salida
    output = layers.Dense(num_classes, activation='softmax', name='output')(x)
    
    # Crear modelo
    model = Model(inputs=[input_img, input_tab], outputs=output, name='MultimodalModel')
    
    return model

# Crear el modelo
print("="*70)
print("CREANDO MODELO MULTIMODAL")
print("="*70)
model = create_multimodal_model(
    img_shape=(224, 224, 3),
    tabular_shape=X_tab_train.shape[1],
    num_classes=num_classes
)
print("="*70)

model.summary()

CREANDO MODELO MULTIMODAL
Intentando cargar EfficientNetV2B3...
✓ EfficientNetV2B3 con pesos de ImageNet cargado exitosamente
Capas entrenables en CNN: 50/410
✓ EfficientNetV2B3 con pesos de ImageNet cargado exitosamente
Capas entrenables en CNN: 50/410


Model: "MultimodalModel"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ tabular_input       │ (None, 21)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ image_input         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tab_dense_1 (Dense) │ (None, 128)       │      2,816 │ tabular_input[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ efficientnetv2-b3   │ (None, 1536)      │ 12,930,622 │ image_input[0][0] │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tab_bn_1            │ (None, 128)       │        512 │ tab_dense_1[0][0] │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ img_dropout_1       │ (None, 1536)      │          0 │ efficientnetv2-b… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tab_dropout_1       │ (None, 128)       │          0 │ tab_bn_1[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ img_dense_1 (Dense) │ (None, 512)       │    786,944 │ img_dropout_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tab_dense_2 (Dense) │ (None, 64)        │      8,256 │ tab_dropout_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ img_bn_1            │ (None, 512)       │      2,048 │ img_dense_1[0][0] │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tab_bn_2            │ (None, 64)        │        256 │ tab_dense_2[0][0] │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ img_dropout_2       │ (None, 512)       │          0 │ img_bn_1[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tab_dropout_2       │ (None, 64)        │          0 │ tab_bn_2[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ img_embedding       │ (None, 256)       │    131,328 │ img_dropout_2[0]… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tab_embedding       │ (None, 128)       │      8,320 │ tab_dropout_2[0]… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fusion              │ (None, 384)       │          0 │ img_embedding[0]… │
│ (Concatenate)       │                   │            │ tab_embedding[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fusion_dense_1      │ (None, 256)       │     98,560 │ fusion[0][0]      │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fusion_bn_1         │ (None, 256)       │      1,024 │ fusion_dense_1[0

 Total params: 14,007,835 (53.44 MB)

 Trainable params: 3,916,571 (14.94 MB)

 Non-trainable params: 10,091,264 (38.50 MB)

In [21]:
# Visualizar arquitectura del modelo
keras.utils.plot_model(
    model,
    to_file='outputs/multimodal_architecture.png',
    show_shapes=True,
    show_layer_names=True,
    rankdir='TB',
    expand_nested=True,
    dpi=96
)

print("Arquitectura del modelo guardada en 'outputs/multimodal_architecture.png'")

You must install pydot (`pip install pydot`) for `plot_model` to work.
Arquitectura del modelo guardada en 'outputs/multimodal_architecture.png'
Arquitectura del modelo guardada en 'outputs/multimodal_architecture.png'


## 6. Compilación y Entrenamiento del Modelo

In [22]:
# Compilar modelo
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=[
        'accuracy',
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc')
    ]
)

print("Modelo compilado exitosamente")

Modelo compilado exitosamente


In [23]:
# Callbacks
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    ),
    ModelCheckpoint(
        'outputs/best_multimodal_model.keras',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

print("Callbacks configurados")

Callbacks configurados


In [24]:
# Entrenar modelo usando los generadores
EPOCHS = 100

print("Iniciando entrenamiento...\n")
print(f"Configuración:")
print(f"  - Épocas máximas: {EPOCHS}")
print(f"  - Batch size: {BATCH_SIZE}")
print(f"  - Batches por época (train): {len(train_generator)}")
print(f"  - Batches por época (val): {len(val_generator)}")
print("="*70)

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)

print("\n" + "="*70)
print("¡Entrenamiento completado!")
print("="*70)

Iniciando entrenamiento...

Configuración:
  - Épocas máximas: 100
  - Batch size: 32
  - Batches por época (train): 933
  - Batches por época (val): 200
Epoch 1/100
Epoch 1/100
933/933 ━━━━━━━━━━━━━━━━━━━━ 0s 513ms/step - accuracy: 0.0716 - auc: 0.5746 - loss: 3.9320 - precision: 0.1203 - recall: 0.0085
Epoch 1: val_accuracy improved from None to 0.48454, saving model to outputs/best_multimodal_model.keras

Epoch 1: val_accuracy improved from None to 0.48454, saving model to outputs/best_multimodal_model.keras
933/933 ━━━━━━━━━━━━━━━━━━━━ 566s 594ms/step - accuracy: 0.1306 - auc: 0.6567 - loss: 3.4948 - precision: 0.2959 - recall: 0.0264 - val_accuracy: 0.4845 - val_auc: 0.9497 - val_loss: 1.7391 - val_precision: 0.8123 - val_recall: 0.1514 - learning_rate: 1.0000e-04
Epoch 2/100
933/933 ━━━━━━━━━━━━━━━━━━━━ 566s 594ms/step - accuracy: 0.1306 - auc: 0.6567 - loss: 3.4948 - precision: 0.2959 - recall: 0.0264 - val_accuracy: 0.4845 - val_auc: 0.9497 - val_loss: 1.7391 - val_precision: 0

KeyboardInterrupt: 

## 7. Visualización del Entrenamiento

In [ ]:
# Graficar curvas de entrenamiento
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Curvas de Entrenamiento del Modelo Multimodal', fontsize=16, fontweight='bold')

metrics = ['loss', 'accuracy', 'precision', 'recall', 'auc']
titles = ['Loss', 'Accuracy', 'Precision', 'Recall', 'AUC']

for idx, (metric, title) in enumerate(zip(metrics, titles)):
    ax = axes[idx // 3, idx % 3]
    ax.plot(history.history[metric], label=f'Train {title}', linewidth=2)
    ax.plot(history.history[f'val_{metric}'], label=f'Val {title}', linewidth=2)
    ax.set_xlabel('Época', fontsize=12)
    ax.set_ylabel(title, fontsize=12)
    ax.set_title(f'{title} vs Época', fontsize=14, fontweight='bold')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)

# Ocultar el subplot vacío
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

## 8. Evaluación en Conjunto de Prueba

In [ ]:
# Predecir en conjunto de prueba usando el generador
print("Generando predicciones en conjunto de prueba...")
y_pred_proba = model.predict(test_generator, verbose=1)
y_pred = np.argmax(y_pred_proba, axis=1)

# Calcular métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

# AUC-ROC (one-vs-rest)
y_test_bin = label_binarize(y_test, classes=range(num_classes))
auc_score = roc_auc_score(y_test_bin, y_pred_proba, average='weighted', multi_class='ovr')

print("="*60)
print("MÉTRICAS EN CONJUNTO DE PRUEBA")
print("="*60)
print(f"Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-Score:  {f1:.4f}")
print(f"AUC-ROC:   {auc_score:.4f}")
print("="*60)

In [ ]:
# Reporte de clasificación detallado
print("\nREPORTE DE CLASIFICACIÓN DETALLADO:\n")
print(classification_report(
    y_test, 
    y_pred, 
    target_names=disease_encoder.classes_,
    zero_division=0
))

## 9. Matriz de Confusión

In [ ]:
# Calcular matriz de confusión
cm = confusion_matrix(y_test, y_pred)

# Visualizar matriz de confusión
plt.figure(figsize=(20, 16))
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    cmap='Blues',
    xticklabels=disease_encoder.classes_,
    yticklabels=disease_encoder.classes_,
    cbar_kws={'label': 'Número de Predicciones'}
)
plt.title('Matriz de Confusión - Modelo Multimodal', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Predicción', fontsize=14, fontweight='bold')
plt.ylabel('Valor Real', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


In [ ]:
# Matriz de confusión normalizada
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(20, 16))
sns.heatmap(
    cm_normalized,
    annot=True,
    fmt='.2f',
    cmap='Blues',
    xticklabels=disease_encoder.classes_,
    yticklabels=disease_encoder.classes_,
    cbar_kws={'label': 'Proporción'},
    vmin=0,
    vmax=1
)
plt.title('Matriz de Confusión Normalizada - Modelo Multimodal', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Predicción', fontsize=14, fontweight='bold')
plt.ylabel('Valor Real', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


## 10. Curvas ROC (One-vs-Rest)

In [ ]:
# Calcular ROC y AUC para cada clase
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(num_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_pred_proba[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Calcular micro-average ROC curve
fpr["micro"], tpr["micro"], _ = roc_curve(y_test_bin.ravel(), y_pred_proba.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

print(f"AUC-ROC Micro-Average: {roc_auc['micro']:.4f}")

In [ ]:
# Visualizar curvas ROC para todas las clases
plt.figure(figsize=(16, 12))

# Línea diagonal (clasificador aleatorio)
plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Clasificador Aleatorio (AUC = 0.50)')

# Curva micro-average
plt.plot(
    fpr["micro"],
    tpr["micro"],
    label=f'Micro-Average (AUC = {roc_auc["micro"]:.3f})',
    color='deeppink',
    linestyle=':',
    linewidth=4
)

# Curvas individuales por clase (mostrar las 10 mejores para claridad)
colors = plt.cm.tab20(np.linspace(0, 1, num_classes))
sorted_classes = sorted(range(num_classes), key=lambda i: roc_auc[i], reverse=True)

for idx, i in enumerate(sorted_classes[:10]):
    plt.plot(
        fpr[i],
        tpr[i],
        color=colors[i],
        lw=2,
        label=f'{disease_encoder.classes_[i]} (AUC = {roc_auc[i]:.3f})'
    )

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos (FPR)', fontsize=14, fontweight='bold')
plt.ylabel('Tasa de Verdaderos Positivos (TPR)', fontsize=14, fontweight='bold')
plt.title('Curvas ROC - Top 10 Clases con Mayor AUC', fontsize=16, fontweight='bold')
plt.legend(loc="lower right", fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('outputs/roc_curves_top10.png', dpi=300, bbox_inches='tight')
plt.show()

print("Curvas ROC guardadas en 'outputs/roc_curves_top10.png'")

In [ ]:
# Visualizar todas las curvas ROC juntas (versión completa)
plt.figure(figsize=(14, 10))

plt.plot([0, 1], [0, 1], 'k--', lw=2, alpha=0.5)

# Todas las curvas con transparencia
for i in range(num_classes):
    plt.plot(fpr[i], tpr[i], lw=1.5, alpha=0.3)

# Destacar micro-average
plt.plot(
    fpr["micro"],
    tpr["micro"],
    label=f'Micro-Average ROC (AUC = {roc_auc["micro"]:.3f})',
    color='red',
    linestyle='-',
    linewidth=3
)

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos (FPR)', fontsize=14, fontweight='bold')
plt.ylabel('Tasa de Verdaderos Positivos (TPR)', fontsize=14, fontweight='bold')
plt.title(f'Curvas ROC - Todas las {num_classes} Clases', fontsize=16, fontweight='bold')
plt.legend(loc="lower right", fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('outputs/roc_curves_all.png', dpi=300, bbox_inches='tight')
plt.show()

print("Curvas ROC completas guardadas en 'outputs/roc_curves_all.png'")

## 11. Análisis de AUC por Clase

In [ ]:
# Crear DataFrame con AUC por clase
auc_df = pd.DataFrame({
    'Enfermedad': disease_encoder.classes_,
    'AUC-ROC': [roc_auc[i] for i in range(num_classes)]
}).sort_values('AUC-ROC', ascending=False)

# Visualizar AUC por clase
plt.figure(figsize=(14, 10))
colors = ['green' if auc >= 0.9 else 'orange' if auc >= 0.8 else 'red' for auc in auc_df['AUC-ROC']]
bars = plt.barh(auc_df['Enfermedad'], auc_df['AUC-ROC'], color=colors, alpha=0.7)

# Añadir líneas de referencia
plt.axvline(x=0.9, color='green', linestyle='--', linewidth=2, alpha=0.5, label='Excelente (≥0.9)')
plt.axvline(x=0.8, color='orange', linestyle='--', linewidth=2, alpha=0.5, label='Bueno (≥0.8)')

# Añadir valores en las barras
for i, (idx, row) in enumerate(auc_df.iterrows()):
    plt.text(row['AUC-ROC'] + 0.01, i, f"{row['AUC-ROC']:.3f}", va='center', fontweight='bold')

plt.xlabel('AUC-ROC', fontsize=14, fontweight='bold')
plt.ylabel('Enfermedad', fontsize=14, fontweight='bold')
plt.title('AUC-ROC por Clase (Ordenado)', fontsize=16, fontweight='bold')
plt.xlim([0, 1.1])
plt.legend(loc='lower right', fontsize=12)
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.savefig('outputs/auc_by_class.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nAUC-ROC por Clase:")
print(auc_df.to_string(index=False))
print(f"\nAUC-ROC Promedio: {auc_df['AUC-ROC'].mean():.4f}")

## 12. Guardar Resultados y Modelo

In [ ]:
# Guardar métricas en CSV
metrics_df = pd.DataFrame({
    'Métrica': ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC-ROC (Micro-Average)'],
    'Valor': [accuracy, precision, recall, f1, roc_auc['micro']]
})

metrics_df.to_csv('outputs/multimodal_metrics.csv', index=False)
auc_df.to_csv('outputs/auc_by_class.csv', index=False)

print("Métricas guardadas en:")
print("  - outputs/multimodal_metrics.csv")
print("  - outputs/auc_by_class.csv")

In [ ]:
# Guardar el modelo final
model.save('outputs/multimodal_model_final.keras')
print("\nModelo final guardado en 'outputs/multimodal_model_final.keras'")

# Guardar el scaler y encoders
import pickle

with open('outputs/preprocessing_objects.pkl', 'wb') as f:
    pickle.dump({
        'scaler': scaler,
        'disease_encoder': disease_encoder,
        'label_encoders': label_encoders,
        'feature_cols': feature_cols,
        'num_classes': num_classes,
        'disease_to_image_class': disease_to_image_class
    }, f)

print("Objetos de preprocesamiento guardados en 'outputs/preprocessing_objects.pkl'")

## 14. Ejemplo de Predicción con el Modelo

In [ ]:
# Función de predicción para nuevas muestras
def predict_disease(model, img_array, tabular_data, disease_encoder, top_k=3):
    """
    Predice la enfermedad para una nueva muestra multimodal
    
    Args:
        model: Modelo entrenado
        img_array: Array de imagen normalizada (224, 224, 3)
        tabular_data: Array de datos tabulares normalizados
        disease_encoder: LabelEncoder de enfermedades
        top_k: Número de predicciones top a retornar
    
    Returns:
        Lista de tuplas (enfermedad, probabilidad)
    """
    # Expandir dimensiones para batch
    img_batch = np.expand_dims(img_array, axis=0)
    tab_batch = np.expand_dims(tabular_data, axis=0)
    
    # Predecir
    predictions = model.predict([img_batch, tab_batch], verbose=0)[0]
    
    # Obtener top-k predicciones
    top_indices = np.argsort(predictions)[-top_k:][::-1]
    top_diseases = disease_encoder.inverse_transform(top_indices)
    top_probs = predictions[top_indices]
    
    return list(zip(top_diseases, top_probs))

# Ejemplo con una muestra del conjunto de prueba
# Cargar una imagen de prueba aleatoria
sample_idx = np.random.randint(0, len(img_paths_test))

# Cargar la imagen
from tensorflow.keras.preprocessing.image import load_img, img_to_array
sample_img = load_img(img_paths_test[sample_idx], target_size=(224, 224))
sample_img = img_to_array(sample_img) / 255.0
sample_tab = X_tab_test[sample_idx]
true_label = disease_encoder.inverse_transform([y_test[sample_idx]])[0]

predictions = predict_disease(model, sample_img, sample_tab, disease_encoder, top_k=5)

print("\nEJEMPLO DE PREDICCIÓN:\n")
print(f"Enfermedad Real: {true_label}")
print("\nTop 5 Predicciones:")
for i, (disease, prob) in enumerate(predictions, 1):
    print(f"  {i}. {disease:30s} - {prob*100:5.2f}%")

# Visualizar la imagen de ejemplo
plt.figure(figsize=(8, 8))
plt.imshow(sample_img)
plt.title(f"Muestra de Prueba\nReal: {true_label} | Predicción: {predictions[0][0]}", 

          fontsize=14, fontweight='bold')
plt.show()
plt.axis('off')
plt.tight_layout()